In [1]:
import sys
from glob import glob

import time
import h5py
import zarr
import numpy as np
import pandas as pd

In [2]:
sys.path.insert(0, '/glade/u/home/ksha/NCAR/')
sys.path.insert(0, '/glade/u/home/ksha/NCAR/libs/')

from namelist import *
import data_utils as du

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
from datetime import datetime, timedelta

In [5]:
import subprocess

In [6]:
lead = 9

### Import HRRR v3

In [7]:
start_time = time.time()

HRRRv3_lead = zarr.load(save_dir_scratch+'HRRR_{:02}_v3.zarr'.format(lead))

print("--- %s seconds ---" % (time.time() - start_time))

--- 1873.171003818512 seconds ---


In [8]:
names = [
    'Maximum/Composite radar reflectivity:dB (instant):lambert:atmosphere:level 0',
    'MSLP (MAPS System Reduction):Pa (instant):lambert:meanSea:level 0',
    'AGL reflectivity (max):lambert:heightAboveGround:level 1000 m',
    'Updraft helicity (max):lambert:heightAboveGroundLayer:levels 5000-2000 m',
    'Updraft helicity (max):lambert:heightAboveGroundLayer:levels 2000-0 m',
    'Updraft helicity (max):lambert:heightAboveGroundLayer:levels 3000-0 m',
    'Vorticity (relative):s**-1 (max):lambert:heightAboveGroundLayer:levels 2000-0 m',
    'Vorticity (relative):s**-1 (max):lambert:heightAboveGroundLayer:levels 1000-0 m',
    'Column integrated graupel mass (max):lambert:atmosphereSingleLayer:level 0 considered as a single layer',
    '2 metre temperature:K (instant):lambert:heightAboveGround:level 2 m',
    '2 metre dewpoint temperature:K (instant):lambert:heightAboveGround:level 2 m',
    '10 metre U wind component:m s**-1 (instant):lambert:heightAboveGround:level 10 m',
    '10 metre V wind component:m s**-1 (instant):lambert:heightAboveGround:level 10 m',
    '10 metre wind speed:m s**-1 (max):lambert:heightAboveGround:level 10 m',
    'Total Precipitation:kg m**-2 (accum):lambert:surface:level 0',
    'Convective available potential energy:J kg**-1 (instant):lambert:surface:level 0',
    'Convective inhibition:J kg**-1 (instant):lambert:surface:level 0',
    'Storm relative helicity:J kg**-1 (instant):lambert:heightAboveGroundLayer:levels 3000-0 m',
    'Storm relative helicity:J kg**-1 (instant):lambert:heightAboveGroundLayer:levels 1000-0 m',
    'Vertical u-component shear:s**-1 (instant):lambert:heightAboveGroundLayer:levels 0-1000 m',
    'Vertical v-component shear:s**-1 (instant):lambert:heightAboveGroundLayer:levels 0-1000 m',
    'Vertical u-component shear:s**-1 (instant):lambert:heightAboveGroundLayer:levels 0-6000 m',
    'Vertical v-component shear:s**-1 (instant):lambert:heightAboveGroundLayer:levels 0-6000 m'
]

In [48]:

with h5py.File(save_dir_scratch+'SPC_to_lead{}.hdf'.format(lead), 'r') as h5io:
    record_v3 = h5io['record_v3'][...]
    
with h5py.File(save_dir+'HRRR_domain.hdf', 'r') as h5io:
    lon_3km = h5io['lon_3km'][...]
    lat_3km = h5io['lat_3km'][...]
    lon_72km = h5io['lon_72km'][...]
    lat_72km = h5io['lat_72km'][...]
    
means = [
    -6.335041783675384,
    101598.30648208999,
    2.4340308170812857,
    0.0238316214287872,
    0.0115228964831135,
    0.015723252607236175,
    0.00010298927478466365,
    0.00013315081911787703,
    0.02022990418418194,
    285.1588453352469,
    280.69456763975046,
    0.18025322895802864,
    -0.35625256772098957,
    4.466962100212334,
    0.10710428466431396,
    311.51020050786116,
    -22.95554152474839,
    95.80303950026172,
    41.22773039479408,
    2.696538199313979,
    0.257023643073863,
    11.80181492281666,
    0.15778718430103703,
];

stds = [
    8.872575669978966,
    672.3339463894478,
    7.555104640235371,
    0.5696550725786566,
    0.2283199203388272,
    0.37333362094670486,
    0.00022281640603195643,
    0.0002413561909874066,
    0.3589573748563584,
    11.553795616392204,
    12.101590155483459,
    3.1758721705443826,
    3.6588052023281175,
    2.6995797278745948,
    0.9896017905552607,
    748.8376068157106,
    78.895180023938,
    104.17948262883918*2,
    77.25788246299936*2,
    5.35086729614372,
    5.438075471238217,
    11.440203318938076,
    11.327741531273508
];

ind_pick = [0, 1, 3, 4, 5, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

log_norm = [True, False, True, True, True, True, True, True, True, False, False, 
            False, False, True, True, True, True, False, False, False, False, False, False]

sparse = [True, False, True, True, True, True, True, True, True, False, False, 
          False, False, False, True, True, True, False, False, False, False, False, False]


grid_shape = record_v3.shape

L_vars = len(ind_pick)

target_size = 24
input_size = 128

half_margin = int((input_size - target_size) / 2)

grid_shape_input = (1059, 1799)

batch_dir = '/glade/scratch/ksha/DATA/NCAR_batch/'
prefix_train = 'TRAIN_pos_{}_mag{}_lead'+str(lead)+'.npy'
prefix_valid = 'VALID_pos_{}_mag{}_lead'+str(lead)+'.npy'

L_train = np.min([675, grid_shape[0]])
L_valid = grid_shape[0]-L_train

count = 0
out_slice = np.empty((1, input_size, input_size, L_vars))

for f in range(int(grid_shape[-1]/3)):
    for i in range(grid_shape[0]):
        
        if i < L_train:
            prefix = prefix_train
        else:
            prefix = prefix_valid
        
        lon_temp = record_v3[i, 3*f]
        lat_temp = record_v3[i, 3*f+1]
        mag_temp = record_v3[i, 3*f+2]
        
        flag_obs = lon_temp + lat_temp

        if np.logical_not(np.isnan(flag_obs)):

            indx_3km, indy_3km = du.grid_search(lon_3km, lat_3km, np.array(lon_temp)[None], np.array(lat_temp)[None])
            indx_3km = indx_3km[0]
            indy_3km = indy_3km[0]

            for augx in range(target_size):
                for augy in range(target_size):

                    x_margin_left = augx
                    x_margin_right = target_size - augx

                    y_margin_bottom = augy
                    y_margin_top = target_size - augy

                    x_edge_left = indx_3km - x_margin_left - half_margin
                    x_edge_right = indx_3km + x_margin_right + half_margin

                    y_edge_bottom = indy_3km - y_margin_bottom - half_margin
                    y_edge_top = indy_3km + y_margin_top + half_margin

                    if x_edge_left >= 0 and y_edge_bottom >= 0 and x_edge_right <= grid_shape_input[0] and y_edge_top <= grid_shape_input[1]:

                        for v, ind_var in enumerate(ind_pick):
                            
                            temp = HRRRv3_lead[i, x_edge_left:x_edge_right, y_edge_bottom:y_edge_top, ind_var]
                            
                            if ind_var == 0:
                                temp[temp<0] = 0
                            
                            if log_norm[ind_var]:
                                temp = np.log(np.abs(temp)+1)
                            else:
                                temp = (temp - means[ind_var])/stds[ind_var]
                            
                            out_slice[..., v] = temp
                            
                        if np.sum(np.isnan(out_slice)) > 0:
                            qwertyyuui
                        else:
                            save_name = batch_dir+prefix.format(count, int(mag_temp))
                            print(save_name)
                            np.save(save_name, out_slice)
                            count += 1

                    else:
                        continue;
                        

/glade/scratch/ksha/DATA/NCAR_batch/TRAIN_pos_0_mag0_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/TRAIN_pos_1_mag0_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/TRAIN_pos_2_mag0_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/TRAIN_pos_3_mag0_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/TRAIN_pos_4_mag0_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/TRAIN_pos_5_mag0_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/TRAIN_pos_6_mag0_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/TRAIN_pos_7_mag0_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/TRAIN_pos_8_mag0_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/TRAIN_pos_9_mag0_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/TRAIN_pos_10_mag0_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/TRAIN_pos_11_mag0_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/TRAIN_pos_12_mag0_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/TRAIN_pos_13_mag0_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/TRAIN_pos_14_mag0_lead9.npy
/glade/scratch/ksha/DATA/NCAR_batch/TRAIN_pos_15_m

KeyboardInterrupt: 